### This can be the final functionized version, where running the tests is easy - just change 'n' and 'bound'
### It is fast at sieving, but is not factoring properly - can you take a look at anything that looks off? 
#### Need to only check after B smooth numbers have already been generated

In [560]:
import math
import numpy as np
import sympy
import time 

In [561]:
def calc_time(start, end):
    seconds = end - start
    mins = seconds / 60
    print('Time in seconds: ', str(seconds), ', Time in minutes: ', str(mins))
    return seconds, mins

In [562]:
# Final optimized version 
def get_primes_less_thanB(bound):                      # eratosthenes way
    check_until = int(bound**0.5)
    unmarked = np.array([x for x in range(2, bound)])
    q = 2
    i = 0
    while True:
        u = unmarked[i]
        if u> check_until:               # only check numbers up to sqrt(bound)
            break
        u_idx = np.where(unmarked == u)[0][0]
        max_idx = bound//u
        listof_multiples = [u*i for i in range(q, max_idx+1)]
        u_multiples = np.array(listof_multiples)
        unmarked = np.setdiff1d(unmarked, u_multiples)   # numpy set diff without changing order of elements in list
        q = u
        i +=1
    print("Number of elements less than bound = ", bound, " is: ", len(unmarked))
    return (unmarked)

In [563]:
# Correct optimized version
def check_Bsmooth(primes, num):
    powers_of_primes= []
    num_remaining= num                         
    for prime in primes: 
        i = 0                                   # i = power of prime
        if (num_remaining%prime == 0):
            while (num_remaining%prime ==0):   #keep dividing for each power of the prime
                quotient = num_remaining/prime
                num_remaining = quotient
                i +=1
        powers_of_primes.append(i)
        if num_remaining == 1:     
            diff = len(primes)-len(powers_of_primes)
            if diff>0:
                zeros = [0 for k in range(diff)]
                return powers_of_primes + zeros
            return powers_of_primes
    #print("Number is not B smooth")
    return []

In [564]:
# Correct optimized version
def check_Bsmooth(primes, num):
    powers_of_primes= []
    num_remaining= num                         
    for prime in primes: 
        i = 0                                   # i = power of prime
        if (num_remaining%prime == 0):
            while (num_remaining%prime ==0):   #keep dividing for each power of the prime
                quotient = num_remaining/prime
                num_remaining = quotient
                i +=1
        powers_of_primes.append(i)
        if num_remaining == 1:     
            diff = len(primes)-len(powers_of_primes)
            if diff>0:
                zeros = [0 for k in range(diff)]
                return powers_of_primes + zeros
            return powers_of_primes
    #print("Number is not B smooth")
    return []

In [565]:
def a_congruent_b(a, b, n):
    if a%n==(b%n):
        return True
    return False

In [566]:
def generate_BSsquares(primes, n, multiple):
    exponents_list = []             # Add other fancy ways of finding candidates for xsquare
    xsquares = []           # the x
    x_vals = []
    residues = []           # the y
    iters = 0
    while(True):
       # if iters>100000000:
       #     break
        x = int(math.sqrt(n*multiple)) +1
        xsquare = x**2
        residue = xsquare%n
        exp_vec = check_Bsmooth(primes, residue)
        if exp_vec != []:
            residues.append(residue)   # x^2
            xsquares.append(xsquare)
            exponents_list.append(exp_vec)
            x_vals.append(x)
            print("Found B-smooth square! with exponents: ", exp_vec, " and quadratic residue: ", residue)
        if len(residues) > len(primes)+10 :
            print("Breaking, len of residues: ", len(residues), " len of primes: ", len(primes))
            break
        multiple += 1
        iters += 1
    return [exponents_list, residues, iters, xsquares, x_vals]

In [567]:
def drop_zero_rows(A): 
    sums = np.sum(A, axis = 1)
    evens = []
    
    for i in range(0, len(sums)):
        if(sums[i]==0):
            evens.append(i)
    
    
    return [A[~np.all(A == 0, axis=1)], evens]

In [568]:
def get_product(v, num_list):  # just multiply all RELEVANT (depends on v) numbers in a list together
    product= 1       
    for i in range(len(v)):
        if v[i] ==0:
            continue
        product = product * num_list[i]
#         product = product%n    # not reducing mod n yet
    return product

In [569]:
def get_root_modN(square, n): # xsq is the number left after multiplying all RELEVANT x squares found, check next func
    result = int(math.sqrt(square))%n
    return result

In [570]:
# Choose a subset of columns (equations) to be 1 - those with 0 in pivot or 1 but the rowsum adds to 0mod2
def getA_solution(L):
    #L = reduced_matrix
    col_num = len(L[0])
    row_num = len(L)
    v = np.ones(col_num, dtype = 'int')
    for i in reversed(range(row_num)):
        for j in range(col_num):
            if L[i][j] ==1:
                if (L[i].sum()%2) !=0:                     # the exponents for this prime doesn't add up to zero
                    v[j] = 0                  # can't use this equation
                    break     # no need to  check cells above, as reduced echelon form
    return(v)


In [571]:
# L - row reduced matrix, comb_iterator - this is for the case when we have reached a factor of gcd and the number itslef
# so we can continue to the next possible combination
def set_v_comb_array(aug_matrix):
    # first set v go have all 0s for pivot nums, 1 for free nums
    col_num = len(aug_matrix[0])
    row_num = len(aug_matrix)
    v = np.ones(col_num, dtype = 'int')
    
    for i in range(0, row_num):
        check_main = 1
        for j in range(i, col_num):
            if(aug_matrix[i][j]==1 and check_main ==1):
                v[j]=0
                break
    
    count_free = np.sum(v); # gives us the count of the free variables.
    v_combs = np.zeros(count_free, dtype = 'int')
    v_combs[count_free-1] = 1;
    indices_of_free_vars = np.zeros(count_free, dtype= 'int')
    
    k=0;
    for i in range(0, col_num):
        if(v[i]==1):
            indices_of_free_vars[k] = i;
            k+=1;
    
    
# v has 1s for free variables and 0s for non-free variables
# v_combs has the current combination of free variables, the initial free variable set to 1 is the last one.
    return [v_combs, indices_of_free_vars]

In [572]:
#input - the matrix, the v vector, and the combination of free variables
#output - a vector that can be used to calculate x, y. 0s for the variables that are not used, 1s for the variables that are used.

def calc_v(L, v_combs, v_indices, rows, cols):
    # create r, such that the only 1s are the free variables specified by v_combs
    r = np.zeros(cols, dtype ='int')
    for i in range(0, len(v_combs)):
        if(v_combs[i]==1):
            r[v_indices[i]] = 1;
        else:
            r[v_indices[i]] =2;    
    
    for i in reversed(range(0, rows)):
    #for j in range(0, col_num):
        sum =0
        for j in range(0, cols):
            sum = sum + r[j]*L[i][j];
        zer =(np.argwhere(r == 0).flatten())
        sum = sum%2;
        if(len(zer)!=0):
            index = zer[len(zer)-1]
    
            if(sum==0):
                r[index] = 2
            else:
                r[index] = 1

    print("final r vector is ", r%2)
    return r%2

In [573]:
def binary_add_arr(comb_arr):
    #for i in reverse(range(0, len(comb_arr))):
    i = len(comb_arr) -1
    while(comb_arr[i]==1):
        comb_arr[i] = 0;
        i-=1;
    if(i>=0):
        comb_arr[i] = 1;
    else:
        return -1
    return comb_arr        

#### Choose the bound value

#### Test: Choose an N of your liking (8-15 digits)

In [574]:
n = 95775679 
k = math.log(n)
k = k*math.log(k)
k = math.exp(math.sqrt(k))
k = math.pow(k, math.sqrt(2)/4)
bound = int(k)

primes = np.array(get_primes_less_thanB(bound))

Number of elements less than bound =  13  is:  5


In [575]:
# A test on a small prime!
# n = 10710336959293
    
# n = 1147
#n = 1147
multiple = 1
start = time.time()

results = generate_BSsquares(primes, n, multiple)

end = time.time()
calc_time(start, end)
print("iterations, ", results[2])
print("B smooth numbers found: ", results[4], "\ntotal: ", len(results[1]))

Found B-smooth square! with exponents:  [8, 0, 0, 3, 0]  and quadratic residue:  87808
Found B-smooth square! with exponents:  [1, 0, 1, 0, 2]  and quadratic residue:  1210
Found B-smooth square! with exponents:  [10, 0, 0, 3, 0]  and quadratic residue:  351232
Found B-smooth square! with exponents:  [1, 0, 0, 0, 5]  and quadratic residue:  322102
Found B-smooth square! with exponents:  [4, 0, 1, 1, 1]  and quadratic residue:  6160
Found B-smooth square! with exponents:  [0, 0, 3, 2, 1]  and quadratic residue:  67375
Found B-smooth square! with exponents:  [1, 0, 1, 1, 0]  and quadratic residue:  70
Found B-smooth square! with exponents:  [8, 2, 0, 3, 0]  and quadratic residue:  790272
Found B-smooth square! with exponents:  [0, 0, 4, 3, 0]  and quadratic residue:  214375
Found B-smooth square! with exponents:  [4, 2, 0, 1, 0]  and quadratic residue:  1008
Found B-smooth square! with exponents:  [5, 8, 1, 0, 0]  and quadratic residue:  1049760
Found B-smooth square! with exponents:  [3

### The following uses (results, n) produced from last run cell 

In [576]:
from sympy import *          # Start using np arrays

A = np.array(results[0])      # results = [exponents_list, residues, iters, xsquares]
store_pows = A.copy()
A = (np.transpose(A))%2
print(A)

[[0 1 0 1 0 0 1 0 0 0 1 1 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 1 1 1 0 0 0 1 1 1 0 0 1]
 [1 0 1 0 1 0 1 1 1 1 0 0 0 1 0 0]
 [0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 1]]


In [577]:
col_num = len(A[0])
row_num = len(A)
print("Old num rows and num columns: ", row_num, col_num)
L = Matrix(A)

L = np.array(L.rref()[0])%2
print(L)# reduced row echelon form
L2  = drop_zero_rows(L)[0]        # get rid of primes we don't have to worry about in general
L2 = L2 %2
    

Old num rows and num columns:  5 16
[[1 0 1 0 0 1 1 1 1 1 0 0 0 1 0 1]
 [0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [578]:
# check for a case where many of the rows have disappeared because they are already even
even_arr = drop_zero_rows(L)[1]
print(even_arr)

[4]


In [579]:
print("Reduced row echelon form AND mod 2 is: ")
print(L2)

print("New num rows, num columns: ", len(L2), len(L2[0]))


Reduced row echelon form AND mod 2 is: 
[[1 0 1 0 0 1 1 1 1 1 0 0 0 1 0 1]
 [0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1]]
New num rows, num columns:  4 16


In [580]:
col_num = len(L2[0])
row_num = len(L2)
v_combs, v_indices = set_v_comb_array(L2)
#print(v_indices)

#print(len(v_indices), col_num)
v = calc_v(L2, v_combs, v_indices, row_num, col_num)     # tells you which equations we can choose
#print(v)


final r vector is  [1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1]


In [581]:
# results is of form: [exponents_list, residues, iters, xsquares, x_vals (x before squaring)]
residues = results[1]    # x%n, RHS
xsquares = results[3]

print(residues)
#print(xsquares)
ysq = get_product(v, residues) 
xsq = get_product(v, xsquares)

print("final x squared mod n: ", xsq%n, " final y squared mod n: ", ysq%n) # just a check - they have to be equal



[87808, 1210, 351232, 322102, 6160, 67375, 70, 790272, 214375, 1008, 1049760, 4840, 784000, 1404928, 1288408, 425920]
final x squared mod n:  55455321  final y squared mod n:  55455321


In [582]:
x_vals = results[4]
print(x_vals)
x = get_product(v, x_vals)
y = sqrt(ysq)

while ((x-y)%n==0 or (x+y)%n==0 or(y-x)%n==0):
    v_combs = binary_add_arr(v_combs)
    v = calc_v(L2, v_combs, v_indices, row_num, col_num)
    x = get_product(v, x_vals)
    ysq = get_product(v, residues) 
    y = sqrt(ysq)
    print('inside: ', x, y)
    
print("\n")
print("Found x not congruent y mod n! A Factor!!")  # THIS ISN'T TRUE! GCD(x-y)doesn't return the right value :"(
print(x, y)
print('current combs are: ',v_combs)

[176971, 315757, 353942, 358379, 430495, 455467, 501027, 530913, 532983, 583837, 593599, 631514, 669429, 707884, 716758, 734901]
final r vector is  [0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0]
inside:  256871015282 644204
final r vector is  [1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1]
inside:  14381830210323167591723977890 9777882920960
final r vector is  [1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
inside:  125274939364 351232
final r vector is  [0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1]
inside:  223954114784564580 60712960
final r vector is  [1 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0]
inside:  32179500863821667360648 226265059328
final r vector is  [0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1]
inside:  57527320841292670366895911560 39111531683840
final r vector is  [0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
inside:  211376892753 30800
final r vector is  [1 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1]
inside:  11834681225602489731615622185 467489792000
final r vector is  [0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0]
inside:  54296597048617538051346 19841483200
final r vector is  [1

In [583]:
diff = abs(x-y)

print("x-y is: ", diff)
gcd = math.gcd(diff, n)
print("GCD found is: ", gcd)
print("other factor of n is: ", int(n/gcd))
print(n)

x-y is:  103322208319
GCD found is:  7757
other factor of n is:  12347
95775679


### THE END

In [584]:
176971*430495*734901

55988528696141145

In [585]:
(256871015282-15178240)%n

81241643